In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TextVectorization
from transformers import AutoTokenizer

import numpy as np
import os
import requests
import io
import pandas as pd
import string

from tqdm import tqdm
from PIL import Image

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95 # Change this value as per requirement
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2023-09-24 10:02:02.059828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 10:02:03.811738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-09-24 10:02:07.176987: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:07.235453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:07.236004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:08.820606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:08.821181: I tensorflow/compile

In [2]:
image_shape = (100, 100, 3)


def download_image_buffer(url):
    img = None
    try:
        data = requests.get(url,headers={"user-agent": "I am a valid user, please give me image"}).content
        img = Image.open(io.BytesIO(data))
        img = img.resize(image_shape[:2])
        img = np.array(img, dtype=np.float64)/255
    except:
        pass
    
    return img


def load_img(path, shape):
    img = keras.utils.load_img(path, color_mode="rgb")
    img = img.resize(shape)
    return np.array(img, dtype=np.float64)/255


learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 10
image_size = 100 
patch_size = 5 
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [20]:
sequence_length = 128
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

In [4]:
_max=100000

with open("./datasets/conceptual-12m/cc12m.tsv", "r", encoding="utf-8") as dataset_file:
    urls, texts = [], []

    for i, item in enumerate(tqdm(dataset_file, total=_max)):
        if i == _max:
            break
        url, text = item.split("\t")

        texts.append(text)

image_files = os.listdir("./datasets/conceptual-12m/images")
image_idx = [int(item.split(".")[0]) for item in tqdm(image_files)]

100%|████████████████████████████████████████████████████████████████| 30186/30186 [00:00<00:00, 2212249.66it/s]


In [5]:

captions = []

for i, item in enumerate(image_idx):
    text = texts[item].replace("\n", "")
    captions.append(text)

In [6]:
image_files = np.array(image_files, dtype=str)
captions = np.array(captions, dtype=str)

base_dir = "./datasets/conceptual-12m/images/"

In [21]:
def preprocess(path, text):
    path = path.numpy().decode('utf-8')
    text = text.numpy().decode('utf-8')

    img = load_img(base_dir + path, image_shape[:2])
    tokenized_text = np.array(tokenizer.encode(text, max_length=sequence_length, add_special_tokens=True, padding="max_length", truncation=True))

    return img, tokenized_text

def dict_map(img, tokenized_text):
    return ({
        "image": img,
        "text": tokenized_text[:-1]
    }, tokenized_text[1:])

In [8]:
dataset = (tf.data.Dataset.from_tensor_slices((image_files, captions))
           .shuffle(1000)
           .map(lambda filepath, caption: tf.py_function(preprocess, [filepath, caption], [tf.float64, tf.int32]))
           .map(dict_map)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))

2023-09-24 10:02:10.811998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:10.812640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:10.813263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:10.814141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-24 10:02:10.814639: I tensorflow/compile

In [37]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches_tensor = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches_tensor.shape[-1]
        patches_tensor = tf.reshape(patches_tensor, [batch_size, -1, patch_dims])
        return patches_tensor

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [38]:
class EncoderBlock(layers.Layer):
    def __init__(self, num_heads, embedding_dims, hidden_layers, hidden_units="same", attn_dropout=0.2, ff_dropout=0.2, **kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embedding_dims = embedding_dims
        self.hidden_layers = hidden_layers
        self.hidden_units = hidden_units if hidden_units != "same" else embedding_dims
        self.attn = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=attn_dropout
        )
        self.attn_norm = layers.LayerNormalization()
        self.ff = []

        for _ in range(hidden_layers):
            self.ff.append(layers.Dense(
                self.hidden_units,
                activation=tf.nn.gelu
            ))
            self.ff.append(layers.Dropout(ff_dropout))

        self.ff = keras.Sequential(self.ff)
        self.ff_norm = layers.LayerNormalization()
    
    def call(self, inputs, mask=None):
        attn_output = self.attn(
            query=inputs,
            key=inputs,
            value=inputs
        )

        attn_output_norm = self.attn_norm(inputs + attn_output)

        x = self.ff(attn_output_norm)
        
        ff_output = self.ff_norm(attn_output + x)
        
        return {
            "ff_output": ff_output,
            "attn_output": attn_output
        }

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embedding_dim": self.embedding_dims,
                "num_heads": self.num_heads,
                "ff_hidden_layers": self.hidden_layers,
                "ff_hidden_units": self.hidden_units
            }
        )
        return config

In [39]:
class DecoderBlock(layers.Layer):
    def __init__(self, 
                 num_heads_self, 
                 embedding_dims_self, 
                 hidden_layers, 
                 hidden_units="same", 
                 num_heads_cross="same", 
                 embedding_dims_cross="same", 
                 attn_dropout=0.2, 
                 ff_dropout=0.2, **kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embedding_dims = embedding_dims_self
        self.hidden_layers = hidden_layers
        self.hidden_units = hidden_units if hidden_units != "same" else embedding_dims_self
        self.num_heads_cross = num_heads_cross if num_heads_cross != "same" else num_heads_self
        self.embedding_dims_cross = embedding_dims_cross if embedding_dims_cross != "same" else embedding_dims_self

        self.self_attn = layers.MultiHeadAttention(
            num_heads=num_heads_self, 
            key_dim=embedding_dims_self,
            dropout=attn_dropout,
            name="Self-attn-decoder"
        )
        self.self_norm = layers.LayerNormalization()

        self.cross_attn = layers.MultiHeadAttention(
            num_heads=self.num_heads_cross, 
            key_dim=self.embedding_dims_cross,
            dropout=attn_dropout,
            name="cross-attn-decoder"
        )
        self.cross_norm = layers.LayerNormalization()
        
        self.ff = []

        for _ in range(hidden_layers):
            self.ff.append(layers.Dense(
                self.hidden_units,
                activation=tf.nn.gelu
            ))
            self.ff.append(layers.Dropout(ff_dropout))

        self.ff = keras.Sequential(self.ff)
        self.ff_norm = layers.LayerNormalization()
        self.add = layers.Add()  # instead of `+` to preserve mask
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        self_attn_output = self.self_attn(
            query=inputs, key=inputs, value=inputs, use_causal_mask=True
        )
        self_attn_output = self.self_norm(self.add([inputs, self_attn_output]))

        cross_attn_output = self.cross_attn(
            query=self_attn_output, key=encoder_outputs, value=encoder_outputs,
        )
        cross_attn_output = self.cross_norm(self.add([self_attn_output, cross_attn_output]))

        ff_output = self.ff(cross_attn_output)
        ff_output = self.ff_norm(self.add([ff_output, cross_attn_output]))

        return ff_output
    
    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embedding_dim": self.embedding_dims,
                "embedding_dim_cross": self.embedding_dims_cross,
                "num_heads": self.num_heads,
                "num_heads_cross": self.num_heads_cross,
                "ff_hidden_layers": self.hidden_layers,
                "ff_hidden_units": self.hidden_units
            }
        )
        return config

In [40]:
num_blocks_encoder = 2
num_blocks_decoder = 2
num_heads = 8
embedding_dims = 64
hidden_layers = 2
text_embedding_dims = 128

In [41]:
image_input = layers.Input(shape=image_shape, name="image")
patches = Patches(patch_size)(image_input)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

caption_input = layers.Input(shape=(sequence_length-1), name="text")
encoded_captions = PositionalEmbedding(sequence_length, vocab_size, embedding_dims)(caption_input)


for _ in range(num_blocks_encoder):
    encoded_patches = EncoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_patches)["ff_output"]


for _ in range(num_blocks_decoder):
    encoded_captions = DecoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_captions, encoded_patches)

top_layer = layers.Dense(1024, activation=tf.nn.gelu)(encoded_captions)
top_layer = layers.Dense(vocab_size, activation=tf.nn.softmax)(top_layer)

test_model = Model(inputs=[image_input, caption_input], outputs=top_layer)
test_model.compile(
    keras.optimizers.AdamW(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [32]:
test = dataset.as_numpy_iterator().next()

In [42]:
test_model.predict(dataset)

ValueError: in user code:

    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filedfn2ghou.py", line 13, in tf__call
        patches_tensor = ag__.converted_ca

In [13]:
image_input = layers.Input(shape=image_shape, name="image")
caption_input = layers.Input(shape=(sequence_length), name="text")

patches = Patches(patch_size)(image_input)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

encoded_captions = PositionalEmbedding(sequence_length, vocab_size, embedding_dims)(caption_input)

attn_weights = []

for _ in range(num_blocks_encoder):
    encoded_patches = EncoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_patches)

    attn_weights.append(encoded_patches["attn_output"])
    encoded_patches = encoded_patches["ff_output"]


for _ in range(num_blocks_decoder):
    encoded_captions = DecoderBlock(
        num_heads,
        embedding_dims,
        hidden_layers
    )(encoded_captions, encoded_patches)

# top_layer = layers.Flatten()(encoded_captions)
top_layer = layers.Dense(1024, activation=tf.nn.gelu)(encoded_captions)
top_layer = layers.Dense(vocab_size, activation=tf.nn.softmax)(top_layer)

transformer = Model(inputs=[image_input, caption_input], outputs=top_layer, name="transformer")
transformer.compile(
    keras.optimizers.AdamW(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 patches (Patches)              (None, None, 75)     0           ['image[0][0]']                  
                                                                                                  
 patch_encoder (PatchEncoder)   (None, 400, 64)      30464       ['patches[0][0]']                
                                                                                                  
 text (InputLayer)              [(None, 128)]        0           []                     

In [14]:
history = transformer.fit(dataset)

2023-09-24 09:49:30.969144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [30186]
	 [[{{node Placeholder/_1}}]]
2023-09-24 09:49:30.969634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [30186]
	 [[{{node Placeholder/_0}}]]


ValueError: in user code:

    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/root/anaconda3/envs/tf/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file58u_9tt3.py", line 13, in tf__call
        patches = ag__.converted_call(ag__.ld(t